# Mandatory Assignment 3


## US Jobs and Import Competition from China 
Af: Mathias Nikolaisen(dnt357), Oliver Mattthews(jvc975) & Clara Maj Folkmann(djz279)

### Problem 1

In [11]:
import pandas as pd


data_path = 'A3_X2017S_MetricsI.dta'
df = pd.read_stata(data_path)

# Beregn beskrivende statistik for hver periode
summary_stats_period_0 = df[df['t2'] == 0][['dsL', 'dIPWusch', 'college', 'routine', 'dIPWusmx', 'foreignborn', 'dIPWotch', 'dIPWukch', 'dIPWotmx']].describe()
summary_stats_period_1 = df[df['t2'] == 1][['dsL', 'dIPWusch', 'college', 'routine', 'dIPWusmx', 'foreignborn', 'dIPWotch', 'dIPWukch', 'dIPWotmx']].describe()

# Udskriv summariske statistikker for begge perioder
print("1990-2000 (t2 = 0):")
print(summary_stats_period_0)

print("2000-2007 (t2 = 1):")
print(summary_stats_period_1)

1990-2000 (t2 = 0):
              dsL    dIPWusch     college     routine    dIPWusmx  \
count  662.000000  662.000000  662.000000  662.000000  662.000000   
mean    -0.957021    1.186600   42.097832   28.288479    1.385140   
std      2.240247    1.838066    8.530094    3.383778    1.745306   
min    -19.167072   -0.075351   19.943975   19.991844   -0.472184   
25%     -2.084500    0.248525   35.936215   25.965810    0.200659   
50%     -0.685493    0.744347   42.515060   28.257560    0.819297   
75%      0.379791    1.426852   47.567284   30.532486    1.908813   
max      5.664484   25.405270   62.898552   37.747578   12.595978   

       foreignborn    dIPWotch    dIPWukch    dIPWotmx  
count   662.000000  662.000000  662.000000  662.000000  
mean      3.916187    1.016592    0.728172    0.014953  
std       4.988597    1.083533    1.053982    0.112670  
min       0.384527   -0.467243   -0.687543   -1.519587  
25%       1.341436    0.282036    0.122727    0.000772  
50%       2.1823

In [13]:
import statsmodels.api as sm

y = df['dsL']
X = df[['t2', 'dIPWusch', 'college', 'foreignborn', 'routine']]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    dsL   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     159.9
Date:                Tue, 22 Oct 2024   Prob (F-statistic):          6.08e-133
Time:                        13:15:54   Log-Likelihood:                -2788.1
No. Observations:                1324   AIC:                             5588.
Df Residuals:                    1318   BIC:                             5619.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.0983      0.544      9.371      

### Problem 2

In [3]:
from linearmodels.iv import IV2SLS

y = df['dsL']

X = df[['t2', 'dIPWusch', 'college', 'foreignborn', 'routine']]
Z = df['dIPWotch']
endog = df['dIPWotch']

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()

print(iv_model.summary)


NameError: name 'df' is not defined

### Problem 3

                            OLS Regression Results                            
Dep. Variable:             ln_aarslon   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     114.6
Date:                Sun, 06 Oct 2024   Prob (F-statistic):          1.48e-135
Time:                        14:52:20   Log-Likelihood:                -3961.5
No. Observations:                4800   AIC:                             7937.
Df Residuals:                    4793   BIC:                             7982.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             8.0633      0.128     62.821

## Problem 4

## Problem 5